# Merging and Concatenating Dataframes


In this section, you will merge and concatenate multiple dataframes. Merging is one of the most common operations you will do, since data often comes in various files. 

In our case, we have sales data of a retail store spread across multiple files. We will now work with all these data files and learn to:
* Merge multiple dataframes using common columns/keys using ```pd.merge()```
* Concatenate dataframes using ```pd.concat()```

Let's first read all the data files.

In [1]:
# loading libraries and reading the data
import numpy as np
import pandas as pd

market_df = pd.read_csv("./global_sales_data/market_fact.csv")
customer_df = pd.read_csv("./global_sales_data/cust_dimen.csv")
product_df = pd.read_csv("./global_sales_data/prod_dimen.csv")
shipping_df = pd.read_csv("./global_sales_data/shipping_dimen.csv")
orders_df = pd.read_csv("./global_sales_data/orders_dimen.csv")

### Merging Dataframes Using ```pd.merge()```

There are five data files:
1. The ```market_fact``` table contains the sales data of each order
2. The other 4 files are called 'dimension tables/files' and contain metadata about customers, products, shipping details, order details etc.

If you are familiar with star schemas and data warehouse designs, you will note that we have one fact table and four dimension tables. 


In [2]:
# Already familiar with market data: Each row is an order
market_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56
1,Ord_5406,Prod_13,SHP_7549,Cust_1818,42.27,0.01,13,4.56,0.93,0.54
2,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59
3,Ord_5456,Prod_6,SHP_7625,Cust_1818,2337.89,0.09,43,729.34,14.30,0.37
4,Ord_5485,Prod_17,SHP_7664,Cust_1818,4233.15,0.08,35,1219.87,26.30,0.38


In [3]:
# Customer dimension table: Each row contains metadata about customers
customer_df.head()

,Customer_Name,Province,Region,Customer_Segment,Cust_id
0,MUHAMMED MACINTYRE,NUNAVUT,NUNAVUT,SMALL BUSINESS,Cust_1
1,BARRY FRENCH,NUNAVUT,NUNAVUT,CONSUMER,Cust_2
2,CLAY ROZENDAL,NUNAVUT,NUNAVUT,CORPORATE,Cust_3
3,CARLOS SOLTERO,NUNAVUT,NUNAVUT,CONSUMER,Cust_4
4,CARL JACKSON,NUNAVUT,NUNAVUT,CORPORATE,Cust_5


In [4]:
# Product dimension table
product_df.head()

,Product_Category,Product_Sub_Category,Prod_id
0,OFFICE SUPPLIES,STORAGE & ORGANIZATION,Prod_1
1,OFFICE SUPPLIES,APPLIANCES,Prod_2
2,OFFICE SUPPLIES,BINDERS AND BINDER ACCESSORIES,Prod_3
3,TECHNOLOGY,TELEPHONES AND COMMUNICATION,Prod_4
4,FURNITURE,OFFICE FURNISHINGS,Prod_5


In [5]:
# Shipping metadata
shipping_df.head()

,Order_ID,Ship_Mode,Ship_Date,Ship_id
0,3,REGULAR AIR,20-10-2010,SHP_1
1,293,DELIVERY TRUCK,02-10-2012,SHP_2
2,293,REGULAR AIR,03-10-2012,SHP_3
3,483,REGULAR AIR,12-07-2011,SHP_4
4,515,REGULAR AIR,30-08-2010,SHP_5


In [6]:
# Orders dimension table
orders_df.head()

,Order_ID,Order_Date,Order_Priority,Ord_id
0,3,13-10-2010,LOW,Ord_1
1,293,01-10-2012,HIGH,Ord_2
2,483,10-07-2011,HIGH,Ord_3
3,515,28-08-2010,NOT SPECIFIED,Ord_4
4,613,17-06-2011,HIGH,Ord_5


# Merging Dataframes

Say you want to select all orders and observe the ```Sales``` of the customer segment *Corporate*. Since customer segment details are present in the dataframe ```customer_df```, we will first need to merge it with ```market_df```.


In [7]:
# Merging the dataframes
# Note that Cust_id is the common column/key, which is provided to the 'on' argument
# how = 'inner' makes sure that only the customer ids present in both dfs are included in the result
df_1 = pd.merge(market_df, customer_df, how='inner', on='Cust_id')
df_1.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,Customer_Name,Province,Region,Customer_Segment
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,AARON BERGMAN,ALBERTA,WEST,CORPORATE
1,Ord_5406,Prod_13,SHP_7549,Cust_1818,42.27,0.01,13,4.56,0.93,0.54,AARON BERGMAN,ALBERTA,WEST,CORPORATE
2,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,AARON BERGMAN,ALBERTA,WEST,CORPORATE
3,Ord_5456,Prod_6,SHP_7625,Cust_1818,2337.89,0.09,43,729.34,14.30,0.37,AARON BERGMAN,ALBERTA,WEST,CORPORATE
4,Ord_5485,Prod_17,SHP_7664,Cust_1818,4233.15,0.08,35,1219.87,26.30,0.38,AARON BERGMAN,ALBERTA,WEST,CORPORATE


In [8]:
# Now, you can subset the orders made by customers from 'Corporate' segment
df_1.loc[df_1['Customer_Segment'] == 'CORPORATE', :]

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,Customer_Name,Province,Region,Customer_Segment
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.8100,0.01,23,-30.51,3.60,0.56,AARON BERGMAN,ALBERTA,WEST,CORPORATE
1,Ord_5406,Prod_13,SHP_7549,Cust_1818,42.2700,0.01,13,4.56,0.93,0.54,AARON BERGMAN,ALBERTA,WEST,CORPORATE
2,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.6900,0.00,26,1148.90,2.50,0.59,AARON BERGMAN,ALBERTA,WEST,CORPORATE
3,Ord_5456,Prod_6,SHP_7625,Cust_1818,2337.8900,0.09,43,729.34,14.30,0.37,AARON BERGMAN,ALBERTA,WEST,CORPORATE
4,Ord_5485,Prod_17,SHP_7664,Cust_1818,4233.1500,0.08,35,1219.87,26.30,0.38,AARON BERGMAN,ALBERTA,WEST,CORPORATE
5,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.0200,0.03,23,-47.64,6.15,0.37,AARON BERGMAN,ALBERTA,WEST,CORPORATE
6,Ord_31,Prod_12,SHP_41,Cust_26,14.7600,0.01,5,1.32,0.50,0.36,AARON BERGMAN,NUNAVUT,NUNAVUT,CORPORATE
45,Ord_4768,Prod_12,SHP_6650,Cust_1579,3.8500,0.08,1,-1.36,0.99,0.37,ADAM HART,BRITISH COLUMBIA,WEST,CORPORATE
46,Ord_4690,Prod_2,SHP_6545,Cust_1579,904.1200,0.07,4,28.81,11.79,0.42,ADAM HART,BRITISH COLUMBIA,WEST,CORPORATE
47,Ord_4659,Prod_6,SHP_6493,Cust_1579,1451.5900,0.06,26,435.11,14.30,0.37,ADAM HART,BRITISH COLUMBIA,WEST,CORPORATE


In [2]:
# Example 2: Select all orders from product category = office supplies and from the corporate segment
# We now need to merge the product_df

df_2 = pd.merge(df_1, product_df, how='inner', on='Prod_id')
df_2.head()

NameError: name 'pd' is not defined

In [1]:
# Select all orders from product category = office supplies and from the corporate segment
df_2.loc[(df_2['Product_Category']=='OFFICE SUPPLIES') & (df_2['Customer_Segment']=='CORPORATE'),:]

NameError: name 'df_2' is not defined


Similary, you can merge the other dimension tables - ```shipping_df``` and ```orders_df``` to create a ```master_df``` and perform indexing using any column in the master dataframe.


In [11]:
# Merging shipping_df
df_3 = pd.merge(df_2, shipping_df, how='inner', on='Ship_id')
df_3.shape

(8399, 19)

In [12]:
# Merging the orders table to create a master df
master_df = pd.merge(df_3, orders_df, how='inner', on='Ord_id')
master_df.shape
master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Region,Customer_Segment,Product_Category,Product_Sub_Category,Order_ID_x,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,...,WEST,CORPORATE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36262,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,WEST,CORPORATE,TECHNOLOGY,TELEPHONES AND COMMUNICATION,36262,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-47.64,6.15,0.37,...,WEST,CORPORATE,OFFICE SUPPLIES,PAPER,36262,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,ONTARIO,HOME OFFICE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",37863,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-17.58,3.98,0.56,...,WEST,CONSUMER,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",53026,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW


Similary, you can perform left, right and outer merges (joins) by using the argument ```how = 'left' / 'right' / 'outer'```.

### Concatenating Dataframes

Concatenation is much more straightforward than merging. It is used when you have dataframes having the same columns and want to append them (pile one on top of the other), or having the same rows and want to append them side-by-side.

#### Concatenating Dataframes Having the Same columns

Say you have two dataframes having the same columns, like so:

In [13]:
# dataframes having the same columns
df1 = pd.DataFrame({'Name': ['Aman', 'Joy', 'Rashmi', 'Saif'],
                    'Age': ['34', '31', '22', '33'],
                    'Gender': ['M', 'M', 'F', 'M']}
                  )

df2 = pd.DataFrame({'Name': ['Akhil', 'Asha', 'Preeti'],
                    'Age': ['31', '22', '23'],
                    'Gender': ['M', 'F', 'F']}
                  )
df1

,Name,Age,Gender
0,Aman,34,M
1,Joy,31,M
2,Rashmi,22,F
3,Saif,33,M


In [14]:
df2

,Name,Age,Gender
0,Akhil,31,M
1,Asha,22,F
2,Preeti,23,F


In [15]:
# To concatenate them, one on top of the other, you can use pd.concat
# The first argument is a sequence (list) of dataframes
# axis = 0 indicates that we want to concat along the row axis
pd.concat([df1, df2], axis = 0)

,Name,Age,Gender
0,Aman,34,M
1,Joy,31,M
2,Rashmi,22,F
3,Saif,33,M
0,Akhil,31,M
1,Asha,22,F
2,Preeti,23,F


In [16]:
# A useful and intuitive alternative to concat along the rows is the append() function
# It concatenates along the rows
df1.append(df2)


,Name,Age,Gender
0,Aman,34,M
1,Joy,31,M
2,Rashmi,22,F
3,Saif,33,M
0,Akhil,31,M
1,Asha,22,F
2,Preeti,23,F


#### Concatenating Dataframes Having the Same Rows

You may also have dataframes having the same rows but different columns (and having no common columns). In this case, you may want to concat them side-by-side. For e.g.:

In [17]:
df1 = pd.DataFrame({'Name': ['Aman', 'Joy', 'Rashmi', 'Saif'],
                    'Age': ['34', '31', '22', '33'],
                    'Gender': ['M', 'M', 'F', 'M']}
                  )
df1

,Name,Age,Gender
0,Aman,34,M
1,Joy,31,M
2,Rashmi,22,F
3,Saif,33,M


In [18]:
df2 = pd.DataFrame({'School': ['RK Public', 'JSP', 'Carmel Convent', 'St. Paul'],
                    'Graduation Marks': ['84', '89', '76', '91']}
                  )
df2

,School,Graduation Marks
0,RK Public,84
1,JSP,89
2,Carmel Convent,76
3,St. Paul,91


In [19]:
# To join the two dataframes, use axis = 1 to indicate joining along the columns axis
# The join is possible because the corresponding rows have the same indices
pd.concat([df1, df2], axis = 1)

,Name,Age,Gender,School,Graduation Marks
0,Aman,34,M,RK Public,84
1,Joy,31,M,JSP,89
2,Rashmi,22,F,Carmel Convent,76
3,Saif,33,M,St. Paul,91


Note that you can also use the ```pd.concat()``` method to merge dataframes using common keys, though here we will not discuss that. For simplicity, we have used the ```pd.merge()``` method for database-style merging and ```pd.concat()``` for appending dataframes having no common columns.

#### Performing Arithmetic Operations on two or more dataframes

We can also perform simple arithmetic operations on two or more dataframes. Below are the stats for IPL 2018 and 2017.

In [20]:
# Teamwise stats for IPL 2018
IPL_2018 = pd.DataFrame({'IPL Team': ['CSK', 'SRH', 'KKR', 'RR', 'MI', 'RCB', 'KXIP', 'DD'],
                         'Matches Played': [16, 17, 16, 15, 14, 14, 14, 14],
                         'Matches Won': [11, 10, 9, 7, 6, 6, 6, 5]}
                       )

# Set the 'IPL Team' column as the index to perform arithmetic operations on the other rows using the team as reference
IPL_2018.set_index('IPL Team', inplace = True)
IPL_2018

,Matches Played,Matches Won
IPL Team,,
CSK,16,11
SRH,17,10
KKR,16,9
RR,15,7
MI,14,6
RCB,14,6
KXIP,14,6
DD,14,5


In [21]:
# Similarly, we have the stats for IPL 2017
IPL_2017 = pd.DataFrame({'IPL Team': ['MI', 'RPS', 'KKR', 'SRH', 'KXIP', 'DD', 'GL', 'RCB'],
                         'Matches Played': [17, 16, 16, 15, 14, 14, 14, 14],
                         'Matches Won': [12, 10, 9, 8, 7, 6, 4, 3]}
                       )
IPL_2017.set_index('IPL Team', inplace = True)
IPL_2017

,Matches Played,Matches Won
IPL Team,,
MI,17,12
RPS,16,10
KKR,16,9
SRH,15,8
KXIP,14,7
DD,14,6
GL,14,4
RCB,14,3


In [22]:
# Simply add the two DFs using the add opearator

Total = IPL_2018 + IPL_2017
Total

,Matches Played,Matches Won
IPL Team,,
CSK,NaN,NaN
DD,28.0,11.0
GL,NaN,NaN
KKR,32.0,18.0
KXIP,28.0,13.0
MI,31.0,18.0
RCB,28.0,9.0
RPS,NaN,NaN
RR,NaN,NaN


Notice that there are a lot of NaN values. This is because some teams which played in IPL 2017 were not present in IPL 2018. In addition, there were also new teams present in IPL 2018. We can handle these NaN values by using `df.add()` instead of the simple add operator. Let's see how.

In [23]:
# The fill_value argument inside the df.add() function replaces all the NaN values in the two dataframes w.r.t. each other with zero.
Total = IPL_2018.add(IPL_2017, fill_value = 0)
Total

,Matches Played,Matches Won
IPL Team,,
CSK,16.0,11.0
DD,28.0,11.0
GL,14.0,4.0
KKR,32.0,18.0
KXIP,28.0,13.0
MI,31.0,18.0
RCB,28.0,9.0
RPS,16.0,10.0
RR,15.0,7.0


Also notice how the resultant dataframe is sorted by the index, i.e. 'IPL Team' alphabetically.

In [24]:
# Creating a new column - 'Win Percentage'

Total['Win Percentage'] = Total['Matches Won']/Total['Matches Played']
Total

,Matches Played,Matches Won,Win Percentage
IPL Team,,,
CSK,16.0,11.0,0.687500
DD,28.0,11.0,0.392857
GL,14.0,4.0,0.285714
KKR,32.0,18.0,0.562500
KXIP,28.0,13.0,0.464286
MI,31.0,18.0,0.580645
RCB,28.0,9.0,0.321429
RPS,16.0,10.0,0.625000
RR,15.0,7.0,0.466667


In [25]:
# Sorting to determine the teams with most number of wins. If the number of wins of two teams are the same, sort by the win percentage.

Total.sort_values(by = (['Matches Won', 'Win Percentage']), ascending = False)

,Matches Played,Matches Won,Win Percentage
IPL Team,,,
MI,31.0,18.0,0.580645
KKR,32.0,18.0,0.562500
SRH,32.0,18.0,0.562500
KXIP,28.0,13.0,0.464286
CSK,16.0,11.0,0.687500
DD,28.0,11.0,0.392857
RPS,16.0,10.0,0.625000
RCB,28.0,9.0,0.321429
RR,15.0,7.0,0.466667


Apart from add(), there are also other operator-equivalent mathematical functions that you can use on Dataframes. Below is a list of all the functions that you can use to perform operations on two or more dataframes
-  `add()`: +
-  `sub()`: -
-  `mul()`: *
-  `div()`: /
-  `floordiv()`: //
-  `mod()`: %
-  `pow()`: **